# 导入文件

In [ ]:
data_path = 'data/datav1.csv'  
df = pd.read_csv(data_path)

# 指定了 parse_dates 参数来解析 TIME 列中的日期数据。
df_date = pd.read_csv('data/data.csv', parse_dates=['TIME'])

# 显示数据格式

In [ ]:
pd.set_option('display.max_rows', None)  # 显示所有行
pd.set_option('display.max_columns', None)  # 显示所有列

In [ ]:
# 禁用科学计数法
pd.set_option('display.float_format', '{:.2f}'.format)

In [ ]:
# 保存CSV 文件
df.to_csv('data/data_1prewash.csv', index=False)

# 查看数据的分布情况，overview

In [ ]:
def data_overview(df):
    """提供数据的基本概览"""
    print("\n======== 数据基本信息 ========")
    print(f"数据行数: {df.shape[0]}")
    print(f"数据列数: {df.shape[1]}")
    
    # 数据类型摘要
    dtypes = df.dtypes.value_counts()
    print("\n数据类型分布:")
    for dtype, count in dtypes.items():
        print(f"- {dtype}: {count}列")
    
    # 查看数据前5行
    print("\n数据前5行预览:")
    display(df.head())
    
    # 显示列信息
    print("\n列信息:")
    column_info = pd.DataFrame({
        '数据类型': df.dtypes,
        '非空值数量': df.count(),
        '空值数量': df.isnull().sum(),
        '空值百分比': (df.isnull().sum() / len(df) * 100).round(2),
        '唯一值数量': df.nunique(),
        '唯一值': {column: df[column].unique() for column in df.columns}
    })
    column_info['空值百分比'] = column_info['空值百分比'].astype(str) + '%'
    display(column_info)
    
    # 基本统计信息,结果格式化为两位小数
    print("\n数值列的统计描述:")
    display(df.describe().T.style.format("{:.2f}"))

    # 分类列的统计描述
    cat_columns = df.select_dtypes(include=['object', 'category']).columns
    if not cat_columns.empty:
        print("\n分类列的统计描述:")
        cat_stats = pd.DataFrame({
            '唯一值数量': df[cat_columns].nunique(),
            '最常见值': [df[col].value_counts().index[0] if not df[col].value_counts().empty else None for col in cat_columns],
            '最常见值频次': [df[col].value_counts().iloc[0] if not df[col].value_counts().empty else 0 for col in cat_columns],
            '最常见值占比': [(df[col].value_counts().iloc[0] / df[col].count() * 100).round(2) if not df[col].value_counts().empty else 0 for col in cat_columns]
        })
        cat_stats['最常见值占比'] = cat_stats['最常见值占比'].astype(str) + '%'
        display(cat_stats)

In [ ]:
# 数值型数据分析结果查看
# count: 数据集中元素的数量。
# mean: 数据的平均值。
# std: 标准差，表示数据的分散程度。
# min: 数据中的最小值。
# 25%: 第一四分位数，表示数据中25%的值小于或等于这个值。
# 50%: 中位数，表示数据中50%的值小于或等于这个值。
# 75%: 第三四分位数，表示数据中75%的值小于或等于这个值。
# max: 数据中的最大值

In [ ]:
# 某个数值型数据列的数据分布情况
def show_value_counts(series):
    value_counts = series.value_counts()
    
    print(f"值统计：")
    print(value_counts)
    print(f"\n共有 {len(value_counts)} 个不同的值")
    
    plt.figure(figsize=(10, 8))
    plt.pie(value_counts.values, 
            labels=[f'{label}\n({count}个)' for label, count in zip(value_counts.index, value_counts.values)],
            autopct='%1.1f%%',
            colors=['lightblue', 'lightgreen', 'lightcoral', 'wheat'],  # 自定义颜色
            explode=[0.1] * len(value_counts),  # 扇形分离效果
            shadow=True)  # 添加阴影
    plt.title('数据分布饼图')
    plt.axis('equal')
    plt.show()

# 检查缺失值

In [ ]:
def handle_missing_values(df):
    missing_cols = df.columns[df.isnull().any()].tolist()
    
    if not missing_cols:
        print("数据中没有缺失值，无需处理。")
        return df
    
    print(f"\n======== 缺失值处理 ========")
    print(f"发现以下{len(missing_cols)}列含有缺失值:")
    
    for col in missing_cols:
        missing_pct = (df[col].isnull().sum() / len(df) * 100).round(2)
        print(f"- {col}: {df[col].isnull().sum()}个缺失值 ({missing_pct}%)")
    
    strategies = {
        '删除': '删除含有缺失值的行',
        '均值填充': '对数值列使用均值填充',
        '中位数填充': '对数值列使用中位数填充',
        '众数填充': '对分类列使用众数填充',
        '0填充': '使用0填充缺失值',
        '指定值填充': '使用指定的值填充缺失值',
        '不处理': '保留缺失值不做处理'
    }
    
    print("\n可用的缺失值处理策略:")
    for key, desc in strategies.items():
        print(f"- {key}: {desc}")
    
    strategy = input("\n请选择缺失值处理策略 (默认为不处理): ") or '不处理'
    
    if strategy not in strategies:
        print(f"未知策略: {strategy}，将默认不处理缺失值。")
        return df
    
    df_clean = df.copy()
    
    if strategy == '删除':
        df_clean = df.dropna()
        print(f"已删除含有缺失值的行，数据形状从 {df.shape} 变为 {df_clean.shape}")
    
    elif strategy == '均值填充':
        for col in missing_cols:
            if pd.api.types.is_numeric_dtype(df[col]):
                df_clean[col] = df[col].fillna(df[col].mean())
                print(f"已对列 '{col}' 使用均值 {df[col].mean():.2f} 填充")
            else:
                print(f"列 '{col}' 不是数值类型，跳过均值填充")
    
    elif strategy == '中位数填充':
        for col in missing_cols:
            if pd.api.types.is_numeric_dtype(df[col]):
                df_clean[col] = df[col].fillna(df[col].median())
                print(f"已对列 '{col}' 使用中位数 {df[col].median():.2f} 填充")
            else:
                print(f"列 '{col}' 不是数值类型，跳过中位数填充")
    
    elif strategy == '众数填充':
        for col in missing_cols:
            mode_value = df[col].mode()[0]
            df_clean[col] = df[col].fillna(mode_value)
            print(f"已对列 '{col}' 使用众数 '{mode_value}' 填充")
    
    elif strategy == '0填充':
        for col in missing_cols:
            df_clean[col] = df[col].fillna(0)
            print(f"已对列 '{col}' 使用0填充")
    
    elif strategy == '指定值填充':
        for col in missing_cols:
            fill_value = input(f"请输入用于填充列 '{col}' 的值: ")
            # 尝试转换为原始列的数据类型
            try:
                if pd.api.types.is_numeric_dtype(df[col]):
                    fill_value = float(fill_value)
                df_clean[col] = df[col].fillna(fill_value)
                print(f"已对列 '{col}' 使用 '{fill_value}' 填充")
            except:
                print(f"无法将 '{fill_value}' 转换为合适的类型，跳过此列")
    
    elif strategy == '不处理':
        print("保留缺失值不做处理")
    
    return df_clean

# 检查异常值

In [ ]:
def handle_outliers(df):
    """检测并处理数据中的异常值"""
    print("\n======== 异常值检测与处理 ========")
    
    # 只对数值列进行异常值检测
    numeric_cols = df.select_dtypes(include=np.number).columns.tolist()
    
    if not numeric_cols:
        print("数据中没有数值列，无法检测异常值。")
        return df
    
    print(f"使用IQR方法检测异常值 (数值列: {len(numeric_cols)}列)")
    
    outliers_summary = {}
    for col in numeric_cols:
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        
        outliers = df[(df[col] < lower_bound) | (df[col] > upper_bound)][col]
        outlier_pct = round((len(outliers) / len(df) * 100), 2)
        
        if not outliers.empty:
            outliers_summary[col] = {
                'count': len(outliers),
                'percentage': outlier_pct,
                'lower_bound': lower_bound,
                'upper_bound': upper_bound
            }
    
    if not outliers_summary:
        print("未检测到异常值。")
        return df
    
    print("\n检测到以下列存在异常值:")
    for col, stats in outliers_summary.items():
        print(f"- {col}: {stats['count']}个异常值 ({stats['percentage']}%), 范围: [{stats['lower_bound']:.2f}, {stats['upper_bound']:.2f}]")
    
    # 绘制箱线图来展示异常值
    n_cols = min(3, len(outliers_summary))
    n_rows = (len(outliers_summary) + n_cols - 1) // n_cols
    
    plt.figure(figsize=(n_cols*5, n_rows*4))
    for i, col in enumerate(outliers_summary.keys()):
        plt.subplot(n_rows, n_cols, i+1)
        sns.boxplot(x=df[col])
        plt.title(f"{col}的箱线图")
        plt.tight_layout()
    plt.savefig('images/outliers_boxplot.png')
    plt.show()
    
    strategies = {
        '删除': '删除含有异常值的行',
        '替换为上下界': '将异常值替换为上下界值',
        '替换为均值': '将异常值替换为均值',
        '替换为中位数': '将异常值替换为中位数',
        '不处理': '保留异常值不做处理'
    }
    
    print("\n可用的异常值处理策略:")
    for key, desc in strategies.items():
        print(f"- {key}: {desc}")
    
    strategy = input("\n请选择异常值处理策略 (默认为不处理): ") or '不处理'
    
    if strategy not in strategies:
        print(f"未知策略: {strategy}，将默认不处理异常值。")
        return df
    
    df_clean = df.copy()
    
    if strategy == '删除':
        original_shape = df_clean.shape
        for col in outliers_summary.keys():
            Q1 = df[col].quantile(0.25)
            Q3 = df[col].quantile(0.75)
            IQR = Q3 - Q1
            lower_bound = Q1 - 1.5 * IQR
            upper_bound = Q3 + 1.5 * IQR
            
            df_clean = df_clean[(df_clean[col] >= lower_bound) & (df_clean[col] <= upper_bound)]
        
        print(f"已删除含有异常值的行，数据形状从 {original_shape} 变为 {df_clean.shape}")
    
    elif strategy == '替换为上下界':
        for col in outliers_summary.keys():
            stats = outliers_summary[col]
            lower_bound = stats['lower_bound']
            upper_bound = stats['upper_bound']
            
            # 将小于下界的值替换为下界值
            df_clean.loc[df_clean[col] < lower_bound, col] = lower_bound
            # 将大于上界的值替换为上界值
            df_clean.loc[df_clean[col] > upper_bound, col] = upper_bound
            
            print(f"已将 '{col}' 列的异常值替换为上下界: [{lower_bound:.2f}, {upper_bound:.2f}]")
    
    elif strategy == '替换为均值':
        for col in outliers_summary.keys():
            stats = outliers_summary[col]
            lower_bound = stats['lower_bound']
            upper_bound = stats['upper_bound']
            mean_value = df[col].mean()
            
            # 替换异常值为均值
            mask = (df_clean[col] < lower_bound) | (df_clean[col] > upper_bound)
            df_clean.loc[mask, col] = mean_value
            
            print(f"已将 '{col}' 列的异常值替换为均值: {mean_value:.2f}")
    
    elif strategy == '替换为中位数':
        for col in outliers_summary.keys():
            stats = outliers_summary[col]
            lower_bound = stats['lower_bound']
            upper_bound = stats['upper_bound']
            median_value = df[col].median()
            
            # 替换异常值为中位数
            mask = (df_clean[col] < lower_bound) | (df_clean[col] > upper_bound)
            df_clean.loc[mask, col] = median_value
            
            print(f"已将 '{col}' 列的异常值替换为中位数: {median_value:.2f}")
    
    elif strategy == '不处理':
        print("保留异常值不做处理")
    
    return df_clean

# 删除不需要的列

In [ ]:
def drop_useless_columns(df):
    
    # 丢弃唯一值数量为 1 的列
    unique_counts = df.nunique()
    columns_to_drop = unique_counts[unique_counts == 1].index
    df = df.drop(columns=columns_to_drop)

    # 丢弃 全部为NaN 的列
    columns_to_drop = df.columns[df.isna().all()]
    df = df.drop(columns=columns_to_drop)

    # 丢弃object 类型的列
    object_columns = df.select_dtypes(include=['object']).columns
    df = df.drop(columns=object_columns)

    # 丢弃 datetime64[ns] 类型的列
    datetime_columns = df.select_dtypes(include=['datetime64[ns]']).columns
    df = df.drop(columns=datetime_columns)

    # 丢弃第 5 到第 11 列（索引从 0 开始，第 5 列的索引是 4，第 11 列的索引是 10）。
    df = df.drop(df.columns[4:11], axis=1)
    # 丢弃第 6 到第 10 列（索引从 0 开始，第 6 列的索引是 5，第 10 列的索引是 9）。
    df = df.drop(df.columns[5:10], axis=1)
  
    return df

# 改变列名

In [ ]:
df1 = df1.rename(columns={
    'plc0': 'c_pv_prewash', 
    'plc6': 'c_sp_prewash', 
    'plc1': 'c_pv_regeneration', 
    'plc2': 'c_pv_regeneration_oil_in', 
    'plc3': 'c_pv_regeneration_oil_out', 
    'plc5': 'mv_pv_prewash', 
    'plc4': 'c_pv_rinse',
    'plc7': 'c_sp_rinse',
    'plc8': 'pa_vacuum_dry',
    'plc9': 'kpa_prewash'
    })

# 保存为 CSV 文件
df1.to_csv('data/data_1prewash.csv', index=False)

# 特征变换

对于某些带有数学参数的项进行特征变换

In [ ]:
from sklearn.preprocessing import PowerTransformer, FunctionTransformer
import numpy as np

# 对数变换 - 处理偏斜数据
df['log_列名'] = np.log1p(df['列名'])  # log1p = log(1+x)

# Box-Cox变换 - 使数据更接近正态分布
pt = PowerTransformer(method='box-cox')
df['boxcox_列名'] = pt.fit_transform(df[['列名']])

# 平方根变换
df['sqrt_列名'] = np.sqrt(df['列名'])

# 立方根变换
df['cbrt_列名'] = np.cbrt(df['列名'])

# 特征分箱

In [ ]:
# 等宽分箱
df['等宽分箱'] = pd.qcut(df['列名'], q=10, labels=False)

# 等频分箱
df['等频分箱'] = pd.cut(df['列名'], bins=10, labels=False)

# 自定义分箱
bins = [0, 18, 35, 50, 65, 100]
labels = ['青少年', '青年', '中年', '中老年', '老年']
df['年龄分组'] = pd.cut(df['年龄'], bins=bins, labels=labels)

# 特征交互

In [ ]:
# 创建多项式特征
from sklearn.preprocessing import PolynomialFeatures
poly = PolynomialFeatures(degree=2, include_bias=False)
poly_features = poly.fit_transform(df[['特征1', '特征2']])

# 基本数学运算
df['特征1_乘_特征2'] = df['特征1'] * df['特征2']
df['特征1_加_特征2'] = df['特征1'] + df['特征2']
df['特征1_除_特征2'] = df['特征1'] / df['特征2']

# 数据平衡（对于分类问题）

In [ ]:
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler

# 过采样
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

# 欠采样
under_sampler = RandomUnderSampler(random_state=42)
X_resampled, y_resampled = under_sampler.fit_resample(X, y)

# 类别数据 编码处理

使用情景：
* 如果类别之间有顺序关系（如：小、中、大），可以使用序数编码
* 如果类别之间没有顺序关系，通常使用独热编码
* 如果类别数量很多，可以考虑使用频率编码或目标编码
* 如果需要保持数据的简单性，可以使用标签编码

注意事项：
* 处理缺失值：在编码之前，需要先处理缺失值
* 新类别处理：在测试集中可能出现训练集中没有的类别，需要考虑如何处理
* 维度灾难：使用独热编码时，如果类别太多，可能会导致特征维度剧增

In [ ]:
# 标签编码 (Label Encoding)
from sklearn.preprocessing import LabelEncoder

# 创建编码器
label_encoder = LabelEncoder()

# 假设我们有一列类别数据
categories = ['红色', '蓝色', '绿色', '红色', '蓝色']

# 进行编码
encoded_categories = label_encoder.fit_transform(categories)
# 结果会是: [2, 0, 1, 2, 0]

In [ ]:
# 独热编码 (One-Hot Encoding)
import pandas as pd

# 假设我们有一个DataFrame
df = pd.DataFrame({
    '颜色': ['红色', '蓝色', '绿色', '红色', '蓝色']
})

# 使用pandas的get_dummies进行独热编码
encoded_df = pd.get_dummies(df['颜色'])

# 结果是
#       红色     绿色     蓝色
# 0   True  False  False
# 1  False  False   True
# 2  False   True  False
# 3   True  False  False
# 4  False  False   True

In [ ]:
# 序数编码 (Ordinal Encoding)
from sklearn.preprocessing import OrdinalEncoder

# 创建编码器
ordinal_encoder = OrdinalEncoder()

# 对于有顺序意义的类别（比如：小，中，大）
categories = [['小', '中', '大']]
ordinal_encoder.fit([['小'], ['中'], ['大']])

In [ ]:
# 频率编码 (Frequency Encoding)

def frequency_encode(data):
    # 计算每个类别的频率
    freq_encoding = data.value_counts(normalize=True).to_dict()
    return data.map(freq_encoding)

# 使用示例
df['颜色_freq'] = frequency_encode(df['颜色'])

In [ ]:
# 目标编码 (Target Encoding)
from category_encoders import TargetEncoder

# 创建编码器
target_encoder = TargetEncoder()

# 假设X是特征（类别列），y是目标变量
X_encoded = target_encoder.fit_transform(X, y)
     

In [ ]:
# 转换二元数据为targe编码，保存为target列
from sklearn.preprocessing import LabelEncoder

def target_encoded(df):
    le = LabelEncoder()
    df['target'] = le.fit_transform(df['acnc'])
    print("类别编码对应关系：")
    for i, label in enumerate(le.classes_):
        print(f"{label} -> {i}")
    return df

# 时间序列分析：

趋势分析： 绘制时间序列图，观察生产指标随时间的变化趋势。

季节性分析： 识别数据中的周期性模式，例如每日、每周或每月的波动。

可能需要处理时区

* 如果需要进行时间序列预测，通常需要提取周期性特征
* 如果是做分类任务，可能需要将时间转换为分类特征
* 如果要分析趋势，可能需要使用滑动窗口特征

In [ ]:
## 使用 Pandas 转换时间格式

import pandas as pd

# 将字符串转换为datetime格式
df['时间列'] = pd.to_datetime(df['时间列'])

# 如果数据格式特殊，可以指定格式
df['时间列'] = pd.to_datetime(df['时间列'], format='%Y-%m-%d %H:%M:%S')

In [ ]:
## 提取时间特征

# 假设df['时间列']已经是datetime格式
df['年'] = df['时间列'].dt.year
df['月'] = df['时间列'].dt.month
df['日'] = df['时间列'].dt.day
df['星期'] = df['时间列'].dt.dayofweek  # 0-6，0代表周一
df['小时'] = df['时间列'].dt.hour
df['分钟'] = df['时间列'].dt.minute
df['是否周末'] = df['时间列'].dt.dayofweek.isin([5,6]).astype(int)

In [ ]:
## 时间差计算

# 计算两个时间之间的差值
df['时间差'] = df['时间列'] - df['时间列'].shift(1)

# 转换时间差为具体数值（如天数、小时数等）
df['天数差'] = df['时间差'].dt.days
df['小时差'] = df['时间差'].dt.total_seconds() / 3600

In [ ]:
## 时间周期特征

# 季节编码
df['季节'] = df['时间列'].dt.quarter

# 月份的正弦和余弦转换（循环特征）
df['月份_sin'] = np.sin(2 * np.pi * df['时间列'].dt.month/12)
df['月份_cos'] = np.cos(2 * np.pi * df['时间列'].dt.month/12)

# 小时的周期性特征
df['小时_sin'] = np.sin(2 * np.pi * df['时间列'].dt.hour/24)
df['小时_cos'] = np.cos(2 * np.pi * df['时间列'].dt.hour/24)

In [ ]:
## 时间窗口统计

# 按照时间窗口进行重采样
# 每小时统计
hourly_stats = df.resample('H', on='时间列').mean()

# 每天统计
daily_stats = df.resample('D', on='时间列').mean()

# 每月统计
monthly_stats = df.resample('M', on='时间列').mean()

In [ ]:
# 设置时区
df['时间列'] = df['时间列'].dt.tz_localize('UTC')
# 转换时区
df['时间列'] = df['时间列'].dt.tz_convert('Asia/Shanghai')

In [ ]:
## 滑动窗口特征


# 数据标准化/归一化

# 常用的可视化图表

折线图： 展示变量随时间的变化趋势。

散点图矩阵： 同时展示多个变量之间的两两关系。

箱线图： 比较不同条件下数据的分布和离群点。 

# 单变量分析

分析数值变量： 计算均值、中位数、标准差等描述性统计量。​
- 计算均值、中位数、标准差等描述性统计量，并绘制直方图或箱线图，了解数据分布情况。​

分析分类变量： 计算频率分布，了解各类别的分布情况。
- 计算频率分布，绘制条形图或饼图，了解各类别的分布情况。   

# 双变量分析

数值变量之间： 使用散点图和计算相关系数，探讨变量之间的线性关系。

数值与分类变量之间： 通过箱线图或小提琴图，比较不同类别下数值变量的分布差异。

分类变量之间： 构建列联表，使用堆积条形图展示类别之间的关系。


# 多变量分析：

热力图： 展示多个变量之间的相关性矩阵，识别潜在的相关关系。

主成分分析（PCA）： 将高维数据降维，识别主要影响因素，便于可视化和后续分析。

## 根据相关性系数可视化相关性热力图（Correlation Heatmap）

相关性系数（Correlation Coefficient）的取值范围是[−1,1]，其含义如下：
* 1：完全正相关，表示两列的变化方向完全一致。
* -1：完全负相关，表示两列的变化方向完全相反。
* 0：无相关性，表示两列之间没有线性关系。

热力图的颜色
* 颜色深浅：颜色越深（通常为红色），表示相关性越强；颜色越浅（通常为蓝色），表示相关性越弱。
* 颜色方向：红色通常表示正相关，蓝色通常表示负相关。

对角线
* 热力图的对角线通常是深色的，因为每列与自身的相关性为 1。

解读步骤

* 强相关性：
寻找颜色较深的区域，这些区域对应的列之间相关性较强。
* 弱相关性：
寻找颜色较浅的区域，这些区域对应的列之间相关性较弱。
* 负相关性：
寻找蓝色区域，这些区域对应的列之间负相关。
* 无相关性：
寻找接近白色的区域，这些区域对应的列之间无相关性。

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

def correlation_heat_map(df):
    correlation_matrix = df.corr()
    # 根据结果适量调整图表大小
    plt.figure(figsize=(30, 30))
    sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm')
    plt.title('Correlation Heatmap')
    plt.show()

## 标准化之后做PCA

In [ ]:
# 标准化数据

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

from sklearn.decomposition import PCA

# 初始化 PCA，设置降维后的维度
pca = PCA(n_components=2)  # 降维到 2 维
X_pca = pca.fit_transform(X_scaled)

# 解释 PCA 结果
print("降维后的数据形状:", X_pca.shape)
print("每个主成分解释的方差比例:", pca.explained_variance_ratio_)
print("主成分的方向:\n", pca.components_)

# 可视化降维结果
plt.scatter(X_pca[:, 0], X_pca[:, 1])
plt.xlabel('PC1')
plt.ylabel('PC2')
plt.title('PCA Result')
plt.show()